# 小売店で扱うデータでデータ加工を行う
## 前提条件
Products A - Z の 26 商品を扱う。
集計期間での商品の単価は変わらない。
売り上げシステムは、従業員が入力し、csvとして保存している。
顧客台帳は、店舗の管理者が週次で集計し、Excel で管理している。
## Data の保存場所
- data_nock/data/ch2/uriage.csv
- 

In [2]:
#売り上げデータの読み込み
import pandas as pd 
uriage_data = pd.read_csv("../../data_nock/data/ch2/uriage.csv")
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二


In [3]:
client_data = pd.read_excel("../../data_nock/data/ch2/kokyaku_daicho.xlsx")
client_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


# データの表記ゆれ
日本語にはたくさんの表記揺れがある。
sample: 日付
- `2019-10-10`
- `2019/10/10`
- `10/10/2019`

sample: 名前
佐々木太郎
佐々木 太郎

> 表記揺れは、フォーマットや入力ミスなどの違いから生まれる。
>
> 違うデータとして取り扱ってしまうことがあるため、要注意.
> 生合成を保つため、`データのゆれ`を把握することから始める。


In [4]:
# 商品名について確認する
uriage_data["item_name"].head()

0      商品A
1    商 品 S
2    商 品 a
3      商品Z
4      商品a
Name: item_name, dtype: object

In [5]:
# 商品の値段について確認する。
uriage_data['item_price'].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

In [6]:
# Nock 13 データに揺れがあるまま集計
# 売り上げ履歴から、商品ごとの月合計売り上げを計算する
uriage_data['purchase_data'] = pd.to_datetime(uriage_data['purchase_date'])
# add new column named 'purchase_month'
uriage_data['purchase_month'] = uriage_data['purchase_data'].dt.strftime("%Y%m")
res = uriage_data.pivot_table(
    index='purchase_month', # y-axis に purchase_month
    columns="item_name",    # x-axis に item_name の表を作る
    aggfunc="size",
    fill_value=0)
res.head()

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,0,1,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


## 注意するべき点
- `商品s`　と　`商品S`が同じものを指していそう
- 5 rows * 99 column となっている 
    - 元々は26商品のみのはず。　
    - 揺れが原因で、正しい商品ごとの集計が取れていない。

In [7]:
#　row に item_price を入れて集計してみる
res = uriage_data.pivot_table(
    index='purchase_month',
    columns='item_name',
    values='item_price',
    aggfunc='sum',
    fill_value=0
)
res.head() #分析結果として全く意味のないもの。

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,0,1400,0,0,0,0,0,0,0,0,...,1100,1200,1500,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,0,2400,0,0,...,0,0,0,0,0,1900,2000,2200,0,0
201903,0,0,500,1300,1600,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,2300,0,0,0,0,0,0,0,0,1700,...,0,0,0,0,0,1900,0,0,0,0
201905,0,0,0,0,0,1900,0,0,0,0,...,0,1200,0,0,0,0,0,0,0,2500


# Nock 14
## 商品名の揺れを補正する
補正すること
- スペースの有無
- 半角/全角 


In [8]:
# 現状確認
print(len(pd.unique(uriage_data.item_name))) # remove redundancy of item_name

99


In [9]:
uriage_data['item_name'] = uriage_data['item_name'].str.upper() # upper case
uriage_data['item_name'] = uriage_data['item_name'].str.replace("  ", "") # remove double white-space
uriage_data['item_name'] = uriage_data['item_name'].str.replace(" ", "") # remove singlw white-space
uriage_data.sort_values(by=['item_name'], ascending=True)

,purchase_date,item_name,item_price,customer_name,purchase_data,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,2019-06-13 18:02:34,201906
1748,2019-05-19 20:22:22,商品A,100.0,松川綾女,2019-05-19 20:22:22,201905
223,2019-06-25 08:13:20,商品A,100.0,板橋隆,2019-06-25 08:13:20,201906
1742,2019-06-13 16:03:17,商品A,100.0,小平陽子,2019-06-13 16:03:17,201906
1738,2019-02-10 00:28:43,商品A,100.0,松田浩正,2019-02-10 00:28:43,201902
...,...,...,...,...,...,...
2880,2019-04-22 00:36:52,商品Y,NaN,田辺光洋,2019-04-22 00:36:52,201904
2881,2019-04-30 14:21:09,商品Y,NaN,高原充則,2019-04-30 14:21:09,201904
1525,2019-01-24 10:27:23,商品Y,2500.0,五十嵐春樹,2019-01-24 10:27:23,201901
1361,2019-05-28 13:45:32,商品Y,2500.0,大崎ヒカル,2019-05-28 13:45:32,201905


In [10]:
# Check if this normalization is working as  expected: 
print(pd.unique(uriage_data['item_name']))
print(len(pd.unique(uriage_data['item_name'])))
# len returns 26 products == what we expected 

['商品A' '商品S' '商品Z' '商品V' '商品O' '商品U' '商品L' '商品C' '商品I' '商品R' '商品X' '商品G'
 '商品P' '商品Q' '商品Y' '商品N' '商品W' '商品E' '商品K' '商品B' '商品F' '商品D' '商品M' '商品H'
 '商品T' '商品J']
26


# Nock 15 
## 金額欠損値の補完
このフェーズでやること
- 欠損値が含まれているかいないかの確認

In [11]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_data     False
purchase_month    False
dtype: bool

結果から、item_price に欠損値があることがわかる。

In [12]:
# get list of rows having item_price = Null
flg_is_null = uriage_data['item_price'].isnull()
print(flg_is_null.head())
trgs = uriage_data.loc[flg_is_null, "item_name"].unique()
print(trgs)
for trg in list(uriage_data.loc[flg_is_null, "item_name"].unique()):
    price = uriage_data.loc[(~flg_is_null) & (uriage_data["item_name"] == trg ), "item_price"].max()
    uriage_data['item_price'].loc[(flg_is_null) & (uriage_data["item_name"] == trg)] = price 
uriage_data.head()


0    False
1     True
2     True
3    False
4     True
Name: item_price, dtype: bool
['商品S' '商品A' '商品P' '商品N' '商品W' '商品R' '商品I' '商品L' '商品F' '商品O' '商品B' '商品C'
 '商品V' '商品Q' '商品U' '商品K' '商品T' '商品X' '商品E' '商品M' '商品G' '商品J' '商品D' '商品H'
 '商品Y']


/Users/kaya/.pyenv/versions/3.7.4/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,purchase_date,item_name,item_price,customer_name,purchase_data,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,2019-06-13 18:02:34,201906
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,2019-07-13 13:05:29,201907
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,2019-05-11 19:42:07,201905
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,2019-02-12 23:40:45,201902
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,2019-04-22 03:09:35,201904


In [13]:
#　修正されたかどうかの確認
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_data     False
purchase_month    False
dtype: bool

All False! 
## 金額が正しく補完されたか確認

In [18]:
for trg in list(uriage_data["item_name"].sort_values().unique()):
    print(trg + "の最大額: " + str(uriage_data.loc[uriage_data["item_name"]==trg]["item_price"].max()) 
    + "の最小額:" 
    + str(uriage_data.loc[uriage_data["item_name"]==trg]["item_price"].min(skipna=False))
    )

商品Aの最大額: 100.0の最小額:100.0
商品Bの最大額: 200.0の最小額:200.0
商品Cの最大額: 300.0の最小額:300.0
商品Dの最大額: 400.0の最小額:400.0
商品Eの最大額: 500.0の最小額:500.0
商品Fの最大額: 600.0の最小額:600.0
商品Gの最大額: 700.0の最小額:700.0
商品Hの最大額: 800.0の最小額:800.0
商品Iの最大額: 900.0の最小額:900.0
商品Jの最大額: 1000.0の最小額:1000.0
商品Kの最大額: 1100.0の最小額:1100.0
商品Lの最大額: 1200.0の最小額:1200.0
商品Mの最大額: 1300.0の最小額:1300.0
商品Nの最大額: 1400.0の最小額:1400.0
商品Oの最大額: 1500.0の最小額:1500.0
商品Pの最大額: 1600.0の最小額:1600.0
商品Qの最大額: 1700.0の最小額:1700.0
商品Rの最大額: 1800.0の最小額:1800.0
商品Sの最大額: 1900.0の最小額:1900.0
商品Tの最大額: 2000.0の最小額:2000.0
商品Uの最大額: 2100.0の最小額:2100.0
商品Vの最大額: 2200.0の最小額:2200.0
商品Wの最大額: 2300.0の最小額:2300.0
商品Xの最大額: 2400.0の最小額:2400.0
商品Yの最大額: 2500.0の最小額:2500.0
商品Zの最大額: 2600.0の最小額:2600.0


# Nock 16
## 顧客名の揺れを補正する